## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-18-21-49-46 +0000,nyt,Live Updates: Zelensky and European Leaders Pr...,https://www.nytimes.com/live/2025/08/18/us/tru...
1,2025-08-18-21-42-06 +0000,nyt,‘Like Death Visited Every Home’: A Pakistani V...,https://www.nytimes.com/2025/08/18/world/asia/...
2,2025-08-18-21-42-05 +0000,nypost,Wild video shows mountain lion strolling into ...,https://nypost.com/2025/08/18/us-news/mountain...
3,2025-08-18-21-38-00 +0000,wsj,Opinion | Trump’s Stunning Foreign Policy,https://www.wsj.com/opinion/trumps-stunning-fo...
4,2025-08-18-21-38-00 +0000,wsj,MSNBC to Change Its Name—and Lose the Peacock ...,https://www.wsj.com/business/media/msnbc-to-ch...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,50
2,zelensky,26
122,ukraine,18
435,house,15
44,will,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
219,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...,145
179,2025-08-18-10-27-12 +0000,nypost,Harrowing photo shows teen killed in Russian s...,https://nypost.com/2025/08/18/world-news/russi...,140
172,2025-08-18-11-37-54 +0000,bbc,"'No going into Nato by Ukraine,' says Trump as...",https://www.bbc.com/news/articles/cm21j1ve817o...,134
84,2025-08-18-18-02-57 +0000,nypost,"Trump, Zelensky strike friendly tone as they t...",https://nypost.com/2025/08/18/us-news/trump-ze...,122
170,2025-08-18-12-03-49 +0000,wapo,"What to expect as Zelensky, European leaders m...",https://www.washingtonpost.com/world/2025/08/1...,118


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
219,145,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...
48,67,2025-08-18-19-54-53 +0000,nypost,Putin claims Ukraine should give up Donetsk be...,https://nypost.com/2025/08/18/world-news/putin...
91,47,2025-08-18-17-35-42 +0000,nyt,Air Canada Flight Attendants Remain on Strike ...,https://www.nytimes.com/2025/08/18/world/canad...
10,42,2025-08-18-21-31-41 +0000,startribune,Are you State Fair ready? Here’s what you need...,https://www.startribune.com/mn-state-fair/6014...
46,39,2025-08-18-19-57-31 +0000,nypost,"Jasveen Sangha, ‘Ketamine Queen’ who sold drug...",https://nypost.com/2025/08/18/us-news/ketamine...
132,34,2025-08-18-15-31-36 +0000,bbc,'No-one was expecting this': European leaders ...,https://www.bbc.com/news/articles/c9d0v8ljw3no...
22,30,2025-08-18-21-00-45 +0000,nyt,"Hamas Accepts New Gaza Cease-Fire Proposal, Of...",https://www.nytimes.com/2025/08/18/world/middl...
108,28,2025-08-18-16-55-53 +0000,cbc,"Texas Democrats return home, Trump rails about...",https://www.cbc.ca/news/world/texas-democrats-...
72,27,2025-08-18-18-32-30 +0000,latimes,Newsmax to pay $67 million to settle Dominion ...,https://www.latimes.com/entertainment-arts/bus...
104,26,2025-08-18-17-06-48 +0000,nypost,Former Vatican chef trades crosses for crosswa...,https://nypost.com/2025/08/18/lifestyle/former...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
